## 10-fold Cross Validation on the Stanik dataset

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import matthews_corrcoef

# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 14.5 MB/s 
     |████████████████████████████████| 5.5 MB 91.4 MB/s 
     |████████████████████████████████| 10.3 MB 53.2 MB/s 
     |████████████████████████████████| 1.3 MB 59.1 MB/s 
     |████████████████████████████████| 1.9 MB 78.2 MB/s 
     |████████████████████████████████| 451 kB 86.9 MB/s 
     |████████████████████████████████| 7.6 MB 74.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 182 kB 75.8 MB/s 
     |████████████████████████████████| 182 kB 82.6 MB/s 
     |████████████████████████████████| 168 kB 84.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 168 kB 91.8 MB/s 
     |████████████████████████████████| 166 kB 55.8 MB/s 
     |████████████████████████████████| 166 kB 89.7 MB/s 
     |████████████████████████████

In [ ]:
# Import downsampled dataset
df = pd.read_excel('DATASET_downsampled_test4.xlsx')

df = df.drop(columns = ['Unnamed: 0'])

df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       2592 non-null   object
 1   kano_labels  2592 non-null   object
 2   labels       2592 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 60.9+ KB


In [ ]:
# Show the distribution of the labels
print(df['labels'].value_counts())

1    648
2    648
3    648
0    648
Name: labels, dtype: int64


In [ ]:
# Rename the columns so that simple transformers can handle them
train_data = df[['review','labels']]
train_data.columns = ['text', 'labels']
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2592 non-null   object
 1   labels  2592 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 40.6+ KB


In [ ]:
# Model implementation and k-fold cross validation with Simpletransformers

# Wrapper function to calculate f1-score with additional parameters
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average = None)

# Wrapper function to calculate recall with additional parameters
def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average = None)

# Wrapper function to calculate precision with additional parameters
def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average = None)

# K-fold cross validation with 10 folds
n = 10
kf = KFold(n_splits=n, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc = []
resultsf1 = []
resultsPrecision = []
resultsRecall = []

model_args = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create the ClassificationModel
model = ClassificationModel(
    'albert',
    'albert-base-v2',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf.split(train_data):
  train_df = train_data.iloc[train_index]
  val_df = train_data.iloc[val_index]

  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)
  print(result)
  
  # Add scores to the respective lists
  resultsAcc.append(result['acc'])
  resultsf1.append(result['f1'])
  resultsPrecision.append(result['precision'])
  resultsRecall.append(result['recall'])

  print("results accuracy",resultsAcc)
  print(f"Mean-Precision: {sum(resultsAcc) / len(resultsAcc)}")
  print('results f1: ', resultsf1)
  print('results precision: ', resultsPrecision)
  print('results recall: ', resultsRecall)

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/292 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

{'mcc': 0.9228611323094305, 'f1': array([0.98360656, 0.92682927, 0.95035461, 0.90909091]), 'acc': 0.9420849420849421, 'recall': array([1.        , 0.91935484, 0.93055556, 0.92307692]), 'precision': array([0.96774194, 0.93442623, 0.97101449, 0.89552239]), 'eval_loss': 0.23240886131922403}
results accuracy [0.7346153846153847, 0.7076923076923077, 0.7181467181467182, 0.806949806949807, 0.8455598455598455, 0.8841698841698842, 0.8841698841698842, 0.918918918918919, 0.9613899613899614, 0.9420849420849421]
Mean-Precision: 0.8403697653697654
results f1:  [array([0.87898089, 0.62385321, 0.84126984, 0.546875  ]), array([0.82706767, 0.624     , 0.82517483, 0.5210084 ]), array([0.83098592, 0.67692308, 0.80916031, 0.52173913]), array([0.88888889, 0.7972973 , 0.848     , 0.703125  ]), array([0.91472868, 0.8115942 , 0.91472868, 0.73770492]), array([0.92413793, 0.86821705, 0.93430657, 0.78504673]), array([0.91603053, 0.89041096, 0.88      , 0.85106383]), array([0.96969697, 0.8976378 , 0.93129771, 0.87

In [ ]:
print("results accuracy",resultsAcc)
print(f"Mean-accuracy: {sum(resultsAcc) / len(resultsAcc)}")
print('results f1: ', resultsf1)
print('results precision: ', resultsPrecision)
print('results recall: ', resultsRecall)

results accuracy [0.7346153846153847, 0.7076923076923077, 0.7181467181467182, 0.806949806949807, 0.8455598455598455, 0.8841698841698842, 0.8841698841698842, 0.918918918918919, 0.9613899613899614, 0.9420849420849421]
Mean-accuracy: 0.8403697653697654
results f1:  [array([0.87898089, 0.62385321, 0.84126984, 0.546875  ]), array([0.82706767, 0.624     , 0.82517483, 0.5210084 ]), array([0.83098592, 0.67692308, 0.80916031, 0.52173913]), array([0.88888889, 0.7972973 , 0.848     , 0.703125  ]), array([0.91472868, 0.8115942 , 0.91472868, 0.73770492]), array([0.92413793, 0.86821705, 0.93430657, 0.78504673]), array([0.91603053, 0.89041096, 0.88      , 0.85106383]), array([0.96969697, 0.8976378 , 0.93129771, 0.875     ]), array([0.97560976, 0.95774648, 0.97478992, 0.94029851]), array([0.98360656, 0.92682927, 0.95035461, 0.90909091])]
results precision:  [array([0.81176471, 0.65384615, 0.9137931 , 0.53846154]), array([0.83333333, 0.6       , 0.81944444, 0.54385965]), array([0.75641026, 0.66666667, 

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

def PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1):

  np_resultsPrecision = np.array(resultsPrecision)
  np_resultsRecall = np.array(resultsRecall)
  np_resultsf1  = np.array(resultsf1)

  # Calculate the mean precision for each label
  df_resultsPrecision = pd.DataFrame(np_resultsPrecision, columns=list('BDIP'))
  precision_b_mean = df_resultsPrecision['B'].mean()
  precision_d_mean = df_resultsPrecision['D'].mean()
  precision_i_mean = df_resultsPrecision['I'].mean()
  precision_p_mean = df_resultsPrecision['P'].mean()

  # Calculate the mean recall for each label
  df_resultsRecall = pd.DataFrame(np_resultsRecall, columns=list('BDIP'))
  recall_b_mean = df_resultsRecall['B'].mean()
  recall_d_mean = df_resultsRecall['D'].mean()
  recall_i_mean = df_resultsRecall['I'].mean()
  recall_p_mean = df_resultsRecall['P'].mean()

  # Calculate the mean f1-score for each label
  df_resultsf1 = pd.DataFrame(np_resultsf1, columns=list('BDIP'))
  f1_b_mean = df_resultsf1['B'].mean()
  f1_d_mean = df_resultsf1['D'].mean()
  f1_i_mean = df_resultsf1['I'].mean()
  f1_p_mean = df_resultsf1['P'].mean()

  # Create dataframe with summarized results
  metrics = [{'B': precision_b_mean, 'D': precision_d_mean, 'I': precision_i_mean, 'P':precision_p_mean},
          {'B': recall_b_mean, 'D': recall_d_mean, 'I': recall_i_mean, 'P': recall_p_mean},
          {'B': f1_b_mean, 'D': f1_d_mean, 'I': f1_i_mean, 'P': f1_p_mean} ]
  train_df_metrics = pd.DataFrame(metrics, index=['Average precision',
                                'Average recall',
                                'Average F1-Score'])
  return train_df_metrics

PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1)

,B,D,I,P
Average precision,0.890555,0.795739,0.902289,0.765072
Average recall,0.934503,0.820491,0.881262,0.717700
Average F1-Score,0.910973,0.807451,0.890908,0.739095


## Testing on the Brunotte dataset

In [ ]:
# Import of 'Trainingskorpus_Final', which gives the final labeling
testset1 = pd.read_excel('Trainingskorpus_Final.xlsx')

testset1 = testset1.drop(columns = ['Unnamed: 0'])
testset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
dtypes: object(2)
memory usage: 25.5+ KB


In [ ]:
# Show the distribution of the labels
print(testset1['kano_labels'].value_counts())

B    1102
P     395
D      95
I      30
Name: kano_labels, dtype: int64


In [ ]:
# Import of 'Trainingskorpus_InitialLabels', which gives the initial labeling of two sometimes differing labels
testset2 = pd.read_excel('Trainingskorpus_InitialLabels.xlsx')

testset2 = testset2.drop(columns = ['Unnamed: 0'])
testset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   1622 non-null   object
 1   label 1  1622 non-null   object
 2   label 2  1622 non-null   object
dtypes: object(3)
memory usage: 38.1+ KB


In [ ]:
# Merge the two sets so we have the final labels as well as the initial labels in the testset
testset = testset1.merge(testset2, left_on='review', right_on='review')
testset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1622 entries, 0 to 1621
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
 2   label 1      1622 non-null   object
 3   label 2      1622 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB


In [ ]:
# Add a column 'equal' which is True if the two initial labels are equal and false otherwise
equal = [None]*np.size(testset['review'])

# Add a column 'equal01' which stores binary values (1 if the two initial labels are equal, 0 otherwise) to calculate the phi-coefficient
equal01 = [None]*np.size(testset['review'])

# Add a column 'labeled_correctly01' which stores binary values (1 if the model correctly labeled the text, 0 otherwise) to calculate the phi-coefficient
labeled_correctly01 = [None]*np.size(testset['review'])

# Add a column 'prediction' which contains the model's predictions
prediction = [None]*np.size(testset['review'])

i = 0
for label1, label2 in zip(testset['label 1'], testset['label 2']):
  if(label1 == label2):
    equal[i] = True
    equal01[i] = 1
  else:
    equal[i] = False
    equal01[i] = 0
  i += 1
  
testset['equal'] = equal

In [ ]:
# Calculate the accuracy of the whole testset as well as 
# only the parts where the inital two labels are equal (accuracy_clear)
# or differ (accuracy_diff)

accuracy_clear = 0.
accuracy_diff = 0.
n_clear = 0
n_diff = 0
all_predicted_labels = []
predicted_labels_equal_initial_label = []
predicted_labels_diff_initial_label = []
true_label_equal = []
true_label_diff = []

# counter to go through all the reviews and fill the column 'labeled_correctly' with true if they have been labeled correctly, and false if not
i = 0

for label, review, equal in zip(testset['kano_labels'],  testset['review'], testset['equal']):
  
  test_prediction, raw_outputs = model.predict([review])
  
  predicted_label = ''

  if test_prediction[0] == 0:
    predicted_label = 'B'
    prediction[i] = 'B'
  if test_prediction[0] == 1:
    predicted_label = 'D'
    prediction[i] = 'D'
  if test_prediction[0] == 2:
    predicted_label = 'I'
    prediction[i] = 'I'
  if test_prediction[0] == 3:
    predicted_label = 'P'
    prediction[i] = 'P'

  all_predicted_labels.append(predicted_label)
  
  if equal:
    n_clear += 1
    predicted_labels_equal_initial_label.append(predicted_label)
    true_label_equal.append(label)
    if label == predicted_label:
      accuracy_clear += 1.

  if not equal:
    n_diff += 1
    predicted_labels_diff_initial_label.append(predicted_label)
    true_label_diff.append(label)
    if label == predicted_label:
      accuracy_diff += 1.

  if label == predicted_label:
    labeled_correctly01[i] = 1
  else:
    labeled_correctly01[i] = 0
  i+=1

testset['labeled_correctly'] = labeled_correctly01
testset['prediction'] = prediction

accuracy = (accuracy_clear + accuracy_diff) / (n_clear + n_diff)

accuracy_clear /= n_clear
accuracy_diff /= n_diff

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
testset.to_excel("Brunotte_with_predictions_test4_albert.xlsx")  

In [ ]:
print('Accuracy Trainingskorpus:                                                  ', round(accuracy*100, 2), '%,  n =', (n_clear + n_diff))
print('Accuracy Trainingskorpus only on the data that coincides on initial labels:', round(accuracy_clear*100, 2), '%,  n =', n_clear)
print('Accuracy Trainingskorpus only on the data that differs on initial labels:  ', round(accuracy_diff*100, 2), '%,  n = ', n_diff)

Accuracy Trainingskorpus:                                                   65.97 %,  n = 1622
Accuracy Trainingskorpus only on the data that coincides on initial labels: 70.28 %,  n = 1383
Accuracy Trainingskorpus only on the data that differs on initial labels:   41.0 %,  n =  239


In [ ]:
# Calculate precision, recall and f1-score for each label

test_df_metrics = precision_recall_fscore_support(testset['kano_labels'], all_predicted_labels, average=None,labels=['B', 'D', 'I', 'P'])

print('Precision B: ', test_df_metrics[0][0])
print('Precision D: ', test_df_metrics[0][1])
print('Precision I: ', test_df_metrics[0][2])
print('Precision P: ', test_df_metrics[0][3])

print('Recall B: ', test_df_metrics[1][0])
print('Recall D: ', test_df_metrics[1][1])
print('Recall I: ', test_df_metrics[1][2])
print('Recall P: ', test_df_metrics[1][3])

print('F1-score B: ', test_df_metrics[2][0])
print('F1-score D: ', test_df_metrics[2][1])
print('F1-score I: ', test_df_metrics[2][2])
print('F1-score P: ', test_df_metrics[2][3])

Precision B:  0.8971098265895954
Precision D:  0.3073770491803279
Precision I:  0.3333333333333333
Precision P:  0.42745098039215684
Recall B:  0.7041742286751361
Recall D:  0.7894736842105263
Recall I:  0.03333333333333333
Recall P:  0.5518987341772152
F1-score B:  0.7890188103711234
F1-score D:  0.4424778761061947
F1-score I:  0.0606060606060606
F1-score P:  0.48176795580110493


In [ ]:
# Calculates the phi-coefficient of the labels being predicted correctly and the researchers being 

print('MCC for initial agreement of raters and correct or false labeling',matthews_corrcoef(labeled_correctly01, equal01))

MCC for initial agreement of raters and correct or false labeling 0.21902173959846843


## 10-fold Cross Validation on a combination of the Stanik dataset and the Brunotte dataset

In [ ]:
# Import combined downsampled dataset 
df_combined = pd.read_excel('DATASET_Trainingskorpus_combined_test4.xlsx')

df_combined = df_combined.drop(columns = ['Unnamed: 0'])

# Rename column so that simple transformers can handle it
df_combined = df_combined.rename(columns = {'review': 'text'})
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         2972 non-null   object
 1   kano_labels  2972 non-null   object
 2   labels       2972 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 69.8+ KB


In [ ]:
# Show the distribution of the labels
print(df_combined['labels'].value_counts())

0    743
2    743
1    743
3    743
Name: labels, dtype: int64


In [ ]:
# 10-fold cross validation on the combined dataset

n2 = 10
kf2 = KFold(n_splits=n2, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc2 = []
resultsf12 = []
resultsPrecision2 = []
resultsRecall2 = []

model_args2 = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create a ClassificationModel
model2 = ClassificationModel(
    'albert',
    'albert-base-v2',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf2.split(df_combined):
  train_df2 = df_combined.iloc[train_index]
  val_df2 = df_combined.iloc[val_index]

  model2.train_model(df_combined)
  result2, model_outputs2, wrong_predictions2 = model2.eval_model(val_df2, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)

  print(result2)
  # Add scores to the respective lists
  resultsAcc2.append(result2['acc'])
  resultsf12.append(result2['f1'])
  resultsPrecision2.append(result2['precision'])
  resultsRecall2.append(result2['recall'])

  print("results accuracy",resultsAcc2)
  print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
  print('results f1: ', resultsf12)
  print('results precision: ', resultsPrecision2)
  print('results recall: ', resultsRecall2)


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

...

{'mcc': 0.9506569202961894, 'f1': array([0.97435897, 0.95541401, 0.97014925, 0.95238095]), 'acc': 0.9629629629629629, 'recall': array([0.96202532, 0.94936709, 0.97014925, 0.97222222]), 'precision': array([0.98701299, 0.96153846, 0.97014925, 0.93333333]), 'eval_loss': 0.3832199518617831}
results accuracy [0.697986577181208, 0.761744966442953, 0.7777777777777778, 0.835016835016835, 0.9057239057239057, 0.8922558922558923, 0.9393939393939394, 0.9393939393939394, 0.9629629629629629]
Mean-accuracy: 0.8569174217943794
results f1:  [array([0.79738562, 0.65060241, 0.88135593, 0.3       ]), array([0.83783784, 0.7654321 , 0.84415584, 0.57575758]), array([0.88757396, 0.72108844, 0.8358209 , 0.65277778]), array([0.87301587, 0.81045752, 0.9352518 , 0.75      ]), array([0.93081761, 0.90066225, 0.94193548, 0.8372093 ]), array([0.91304348, 0.89189189, 0.92957746, 0.84337349]), array([0.97222222, 0.90277778, 0.98630137, 0.9       ]), array([0.98734177, 0.91156463, 0.94890511, 0.90789474]), array([0.9743

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.950762354329458, 'f1': array([0.98159509, 0.95104895, 0.97368421, 0.94117647]), 'acc': 0.9629629629629629, 'recall': array([0.97560976, 0.97142857, 0.98666667, 0.91428571]), 'precision': array([0.98765432, 0.93150685, 0.96103896, 0.96969697]), 'eval_loss': 0.1689380203422747}
results accuracy [0.697986577181208, 0.761744966442953, 0.7777777777777778, 0.835016835016835, 0.9057239057239057, 0.8922558922558923, 0.9393939393939394, 0.9393939393939394, 0.9629629629629629, 0.9629629629629629]
Mean-accuracy: 0.8675219759112377
results f1:  [array([0.79738562, 0.65060241, 0.88135593, 0.3       ]), array([0.83783784, 0.7654321 , 0.84415584, 0.57575758]), array([0.88757396, 0.72108844, 0.8358209 , 0.65277778]), array([0.87301587, 0.81045752, 0.9352518 , 0.75      ]), array([0.93081761, 0.90066225, 0.94193548, 0.8372093 ]), array([0.91304348, 0.89189189, 0.92957746, 0.84337349]), array([0.97222222, 0.90277778, 0.98630137, 0.9       ]), array([0.98734177, 0.91156463, 0.94890511, 0.907894

In [ ]:
print("results accuracy",resultsAcc2)
print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
print('results f1: ', resultsf12)
print('results precision: ', resultsPrecision2)
print('results recall: ', resultsRecall2)

results accuracy [0.697986577181208, 0.761744966442953, 0.7777777777777778, 0.835016835016835, 0.9057239057239057, 0.8922558922558923, 0.9393939393939394, 0.9393939393939394, 0.9629629629629629, 0.9629629629629629]
Mean-accuracy: 0.8403697653697654
results f1:  [array([0.79738562, 0.65060241, 0.88135593, 0.3       ]), array([0.83783784, 0.7654321 , 0.84415584, 0.57575758]), array([0.88757396, 0.72108844, 0.8358209 , 0.65277778]), array([0.87301587, 0.81045752, 0.9352518 , 0.75      ]), array([0.93081761, 0.90066225, 0.94193548, 0.8372093 ]), array([0.91304348, 0.89189189, 0.92957746, 0.84337349]), array([0.97222222, 0.90277778, 0.98630137, 0.9       ]), array([0.98734177, 0.91156463, 0.94890511, 0.90789474]), array([0.97435897, 0.95541401, 0.97014925, 0.95238095]), array([0.98159509, 0.95104895, 0.97368421, 0.94117647])]
results precision:  [array([0.72619048, 0.54      , 0.92857143, 0.5       ]), array([0.80519481, 0.70454545, 0.83333333, 0.69090909]), array([0.88235294, 0.74647887, 0

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

PrecisionRecallF1Table(resultsPrecision2, resultsRecall2, resultsf12)

,B,D,I,P
Average precision,0.900857,0.838094,0.934736,0.790625
Average recall,0.932308,0.863077,0.915655,0.758508
Average F1-Score,0.915519,0.846094,0.924714,0.766057
